In [1]:
pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     -------------------------------------- 129.4/129.4 kB 7.4 MB/s eta 0:00:00
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: bs4 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [2]:
import pandas as pd
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
import time
import re


In [60]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--disable-extensions')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-browser-side-navigation')
options.add_argument('--disable-gpu-sandbox')
browser = webdriver.Chrome(options=options)
url = 'https://www.target.com/c/women/-/N-5xtd3?Nao=1&moveTo=product-list-grid'
def get_page(url):
    browser.get(url)
    time.sleep(4)
    return browser
get_page(url)

<selenium.webdriver.chrome.webdriver.WebDriver (session="836d51bc469d810836a48eebbea4fc6d")>

In [43]:
def go_to_next_page(browser):
    current_url = browser.current_url
    current_nao = int(re.search(r'Nao=(\d+)', current_url).group(1))
    new_nao = current_nao + 24
    next_url = re.sub(r'Nao=\d+', f'Nao={new_nao}', current_url)
    browser.get(next_url)

In [59]:
def scroll_to_bottom(browser):
    # Scroll down in increments of 500 pixels
    prev_height = browser.execute_script("return window.pageYOffset;")
    height = browser.execute_script("return document.body.scrollHeight")
    for i in range(prev_height, height-500, 500):
        browser.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.5)
scroll_to_bottom(browser)

In [20]:
def scrape_product_names(browser):
    product_names = []
    product_elements = browser.find_elements(By.XPATH, "//a[@data-test='product-title']")
    if product_elements is not None:
        for element in product_elements:
            try:
                product_names.append(element.text)
            except:
                time.sleep(2)
                product_names.append(element.text)
    return product_names

In [21]:
def scrape_product_prices(browser):
    product_prices = []
    product_elements = browser.find_elements(By.XPATH, "//span[@data-test='product-price']")
    for element in product_elements:
        try:
            product_prices.append(element.text)
        except:
            time.sleep(2)
            product_prices.append(element.text)
    return product_prices

In [62]:
product_names = []
product_prices = []
def get_all_items(browser):
    page_num = 1
    while page_num <= 50:
        print(f"Scraping page {page_num}")
        scroll_to_bottom(browser)
        time.sleep(2)
        product_names.extend(scrape_product_names(browser))
        product_prices.extend(scrape_product_prices(browser))
        go_to_next_page(browser)
        time.sleep(2)
        page_num += 1
    return product_names, product_prices

get_all_items(browser)

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50


(["Women's High-Waisted Classic Leggings - Wild Fable™ Black",
  "Women's Tiny Tank Top - Wild Fable™",
  "90 Degree by Reflex - Women's Soft Comfy Lounge Shorts with Pockets",
  "Women's Tonal Shamrocks St. Patrick's Day Crew Socks - Dark Green 4-10",
  "Women's Short Sleeve Casual T-Shirt - A New Day™",
  'Plush Cotton Unisex Adult Bathrobe - Blue Nile Mills',
  "Women's Gradient Shamrock St. Patrick's Day Low Cut Socks - Black 4-10",
  "Women's Short Sleeve Ribbed T-Shirt - A New Day™",
  "Alexander del Rossa Women's Raglan Sleep Shirt with Pockets, Long Sleeve Nightshirt, Lightweight Nightgown",
  "Women's Short Sleeve Extended Shoulder T-Shirt - A New Day™",
  "Women's Short Sleeve T-Shirt - Wild Fable™",
  "Alexander del Rossa Women's Raglan Sleep Shirt, Short Sleeve Nightshirt, Lightweight Nightgown",
  "Women's Long Sleeve Relaxed Fit Collared Button-Down Shirt - Universal Thread™",
  "Fashion Forms Women's Reusable Silicone Gel Petals",
  "Women's Slim Fit Square Neck Tank Top

In [66]:
cleaned_names = []
def clean_product_names(product_names):
    for name in product_names:
        # Use regular expressions to remove "Women's" from the beginning of the name
        cleaned_name = re.sub(r"^Women's\s+", "", name)
        cleaned_names.append(cleaned_name)
    return cleaned_names
clean_product_names(product_names)

['High-Waisted Classic Leggings - Wild Fable™ Black',
 'Tiny Tank Top - Wild Fable™',
 "90 Degree by Reflex - Women's Soft Comfy Lounge Shorts with Pockets",
 "Tonal Shamrocks St. Patrick's Day Crew Socks - Dark Green 4-10",
 'Short Sleeve Casual T-Shirt - A New Day™',
 'Plush Cotton Unisex Adult Bathrobe - Blue Nile Mills',
 "Gradient Shamrock St. Patrick's Day Low Cut Socks - Black 4-10",
 'Short Sleeve Ribbed T-Shirt - A New Day™',
 "Alexander del Rossa Women's Raglan Sleep Shirt with Pockets, Long Sleeve Nightshirt, Lightweight Nightgown",
 'Short Sleeve Extended Shoulder T-Shirt - A New Day™',
 'Short Sleeve T-Shirt - Wild Fable™',
 "Alexander del Rossa Women's Raglan Sleep Shirt, Short Sleeve Nightshirt, Lightweight Nightgown",
 'Long Sleeve Relaxed Fit Collared Button-Down Shirt - Universal Thread™',
 "Fashion Forms Women's Reusable Silicone Gel Petals",
 'Slim Fit Square Neck Tank Top - A New Day™',
 'Elbow Sleeve Boxy Cropped T-Shirt - A New Day™',
 'Short Sleeve V-Neck T-Shir

In [67]:
#add product names and prices to dataframe
df = pd.DataFrame({'product_name': cleaned_names, 'product_price': product_prices})
df

,product_name,product_price
0,High-Waisted Classic Leggings - Wild Fable™ Black,$7.00
1,Tiny Tank Top - Wild Fable™,$4.00
2,90 Degree by Reflex - Women's Soft Comfy Loung...,$19.99
3,Tonal Shamrocks St. Patrick's Day Crew Socks -...,$4.00
4,Short Sleeve Casual T-Shirt - A New Day™,$4.80
...,...,...
1204,U-Neck Multi-Way Bralette Bikini Top - Wild Fa...,$18.00
1205,Tie-Front Dolman Short Sleeve T-Shirt - Ava & ...,$14.40
1206,Super-High Rise Slim Straight Jeans - Wild Fab...,$25.00
1207,Pucker Texture Square Neck Bikini Top - Kona Sol™,$23.00


In [68]:
#save dataframe to csv

df.to_csv('Target_Data.csv', index=True)

In [132]:
#check the length of the list of product names and prices
len(product_names), len(product_prices)


(26, 26)

Found 0 product elements


[]

In [103]:
def scrape_product_names(browser):
    product_names = []
    product_elements = browser.find_elements(By.XPATH, "//a[@data-test='product-title']")
    for element in product_elements:
        try:
            product_names.append(element.text)
        except:
            time.sleep(2)
            product_names.append(element.text)
    return product_names

scrape_product_names(browser)


[]

In [10]:
#call each function to get the data and append it to a list to be used to create a dataframe
products = []

def get_data():
    page_number = 1
    scroll_to_bottom(browser)
    while page_number < 2:
        print(f"Scraping page {page_number}...")
        temp_lst = []
        item_brand = get_item_brand(browser)
        item_name = get_item_name(browser)
        item_price = get_item_price(browser)
        temp_lst.append(item_brand + item_name + item_price)
        products.append(temp_lst)
        temp_lst.clear()
        go_to_next_page(browser)
        page_number += 1
        print('Next page...')

get_data() #call the function to get the data

Scraping page 1...


TypeError: list.append() takes exactly one argument (3 given)

In [75]:
#call each function, then append the list for each page to a list that contains all pages
full_clothing_lst = []
full_price_lst=[]
def get_all_pages():
    get_page(url)
    close_popup()
    print("Pop-up Closed")
    page_number = 1
    while page_number <= 2:
        
        page_number += 1
get_all_pages()

Pop-up Closed
Scraping page 1...
Extracted 48 items from page 1.
Scraping page 2...
Extracted 48 items from page 2.
Scraping page 3...
Extracted 48 items from page 3.
Scraping page 4...
Extracted 48 items from page 4.
Scraping page 5...
Extracted 48 items from page 5.
Scraping page 6...
Extracted 48 items from page 6.
Scraping page 7...
Extracted 48 items from page 7.
Scraping page 8...
Extracted 48 items from page 8.
Scraping page 9...
Extracted 48 items from page 9.
Scraping page 10...
Extracted 48 items from page 10.
Scraping page 11...
Extracted 48 items from page 11.
Scraping page 12...
Extracted 48 items from page 12.
Scraping page 13...
Extracted 48 items from page 13.
Scraping page 14...
Extracted 48 items from page 14.
Scraping page 15...
Extracted 48 items from page 15.
Scraping page 16...
Extracted 48 items from page 16.
Scraping page 17...
Extracted 48 items from page 17.
Scraping page 18...
Extracted 48 items from page 18.
Scraping page 19...
Extracted 48 items from page 1

In [74]:
def scrape_page():
    time.sleep(1)
    #create lists for items and prices
    clothing_lst = []
    clothing_lst_website = browser.find_elements(By.XPATH, ("//div[@class='prod_nameBlock']/p")) 
    listed_price_lst = []
    listed_price_lst_website = browser.find_elements(By.CLASS_NAME,'prod_price_amount')    

    #loop through each page then add items and prices to lists
    for thing in range(len(clothing_lst_website)):
        try:
            clothing_lst.append(clothing_lst_website[thing].text)
        except StaleElementReferenceException:
            # if the element is stale, retry the operation
            time.sleep(1)
            clothing_lst_website = browser.find_elements(By.XPATH, ("//div[@class='prod_nameBlock']/p"))
            clothing_lst.append(clothing_lst_website[thing].text)

    for thing in range(len(listed_price_lst_website)):
        try:
            listed_price_lst.append(listed_price_lst_website[thing].text)
        except StaleElementReferenceException:
            # if the element is stale, retry the operation
            time.sleep(1)
            listed_price_lst_website = browser.find_elements(By.CLASS_NAME,'prod_price_amount')
            listed_price_lst.append(listed_price_lst_website[thing].text)

    #clear list for next page
    clothing_lst_website = clothing_lst_website.clear()
    listed_price_lst_website = listed_price_lst_website.clear()
    return clothing_lst, listed_price_lst


In [76]:
#flatten the list of lists
flat_clothing_lst = [item for sublist in full_clothing_lst for item in sublist]
flat_price_lst = [item for sublist in full_price_lst for item in sublist]

#convert to dataframe
df = pd.DataFrame(list(zip(flat_clothing_lst, flat_price_lst)), columns =['Clothing', 'Price'])
df.drop_duplicates(subset="Clothing", keep=False, inplace=True)
df

,Clothing,Price
0,Women's Gloria Vanderbilt Amanda Classic Jeans,$24.99
2,Women's Sonoma Goods For Life® Supersoft Stret...,$12.60 - $24.99
3,Women's Sonoma Goods For Life® Supersoft Midri...,$12.60 - $24.99
4,Women's Sonoma Goods For Life® Midrise Bootcut...,$24.99
5,Women's Sonoma Goods For Life® Straight-Leg ...,$12.60 - $24.99
...,...,...
9595,Bella + Canvas Womens/Ladies Jersey Short-Slee...,$21.99
9596,Women's adidas Floral Logo Long Sleeve T-Shirt,$21.99
9597,"Women's PSK Collective ""Be You, The World Will...",$21.75
9598,Women's PSK Collective Placed Floral Tee,$21.75


In [77]:
#save to csv
df.to_csv('kohls_data_output.csv', index=True)

47

In [11]:
=

[["Women's Gloria Vanderbilt Amanda Classic Jeans",
  "Women's Croft & Barrow® Effortless Stretch Pull-On ...",
  "Women's Sonoma Goods For Life® Supersoft Stretch ...",
  "Women's Sonoma Goods For Life® Supersoft Midrise ...",
  "Women's Sonoma Goods For Life® Midrise Bootcut Jeans",
  "Women's Sonoma Goods For Life® Straight-Leg ...",
  'Plus Size Gloria Vanderbilt Amanda Classic Jeans',
  "Women's Lee® Wrinkle-Free Relaxed Fit Straight-Leg ...",
  "Women's Levi's® Classic Straight-Leg Jeans",
  "Women's Tek Gear® Ultrasoft Fleece Sweatshirt",
  'Bali® Comfort Revolution® ComfortFlex Fit® Shaping ...',
  "Women's Gloria Vanderbilt Amanda Pull-On Jeans",
  "Women's Lee® Relaxed Fit Straight-Leg Jeans",
  "Women's Nine West Pull-On Jeggings",
  "Women's Nine West Relaxed Blazer",
  'Playtex® 18 Hour Ultimate Comfort Strap Full-Figure ...',
  "Juniors' WallFlower Insta Stretch® Luscious Curvy ...",
  "Women's Sonoma Goods For Life® Everyday V-Neck Tee",
  "Women's Sonoma Goods For Life®

In [126]:
browser2 = webdriver.Chrome()
# Example usage
url2 = "https://www.kohls.com/catalog/womens-clothing.jsp?CN=Gender:Womens+Department:Clothing&icid=wo-size-womens&kls_sbp=72721882801325196462708528859614168656"
browser2.get(url2)
time.sleep(2)

In [128]:
def go_to_next_page(browser2):
    next_button = browser2.find_element(By.XPATH, "//a[@class='ce-pgntn nextArw fr']")
    next_button.click()
go_to_next_page(browser2)    

In [43]:
def get_page_url(url, page_number):
    page_suffix = "&page=" + str(page_number)
    return url + page_suffix

In [44]:
browser2 = webdriver.Chrome()
# Example usage
url = "https://www.kohls.com/catalog/womens-clothing.jsp?CN=Gender:Womens+Department:Clothing&icid=wo-size-womens&kls_sbp=72721882801325196462708528859614168656"
time.sleep(2)
# Go to page 2
page_number = 2
url = get_page_url(url, page_number)
browser2.get(url)


In [40]:
def go_to_next_page(browser2):
    try:
        next_page_button = browser2.find_element_by_xpath("//a[@class='ce-pgntn nextArw fr']")
        next_page_button.click()
        time.sleep(2)
    except:
        print("No more pages")


In [22]:
def next_page():
    next_page = browser2.find_element_by_xpath("/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div/div/div[3]/div/div[4]/div[3]/div/a[2]")
    next_page.click()
    time.sleep(5)

In [22]:
url = 'https://www.kohls.com/catalog/womens-clothing.jsp?CN=Gender:Womens+Department:Clothing&icid=wo-size-womens&kls_sbp=72721882801325196462708528859614168656'

browser.get(url)
browser.execute_script("window.scrollBy(0,250);")
page_number = 1

item_dict = {}


def ScrapePage(browser, url_list):
    while page_number < 3:
        clothing_lst = []
        listed_price_lst = []
        #get the names of listings from the website
        clothing_lst_website = browser.find_elements(By.XPATH, ("//div[@class='prod_nameBlock']/p")) 
        for thing in range(len(clothing_lst_website)):
            clothing_lst.append(clothing_lst_website[thing].text)    
        listed_price_lst_website = browser.find_elements(By.CLASS_NAME,'prod_price_amount')   
        for thing in range(len(listed_price_lst_website)):
            listed_price_lst.append(listed_price_lst_website[thing].text)

        for i in range(len(clothing_lst)):
            item_dict[clothing_lst[i]] = listed_price_lst[i]
        try:
            next_page = browser.find_element_by_xpath("//a[@class='ce-pgntn nextArw fr']")
            next_page.click()
            time.sleep(5)
            page_number += 1
            print(page_number)
        except:
            break


In [37]:
item_list = []

def ScrapePage(browser, url):
    page_number = 1
    
    while page_number < 10:
        clothing_lst = []
        listed_price_lst = []
        #get the names of listings from the website
        clothing_lst_website = browser.find_elements(By.XPATH, ("//div[@class='prod_nameBlock']/p")) 
        for thing in range(len(clothing_lst_website)):
            clothing_lst.append(clothing_lst_website[thing].text)    
        listed_price_lst_website = browser.find_elements(By.CLASS_NAME,'prod_price_amount')   
        for thing in range(len(listed_price_lst_website)):
            listed_price_lst.append(listed_price_lst_website[thing].text)

        #add clothing_list and listed_price_list to the global list item_list
        for i in range(len(clothing_lst)):
            item_list.append(clothing_lst[i])
            item_list.append(listed_price_lst[i])

        try:
            next_page = browser.find_element_by_xpath("//a[@class='ce-pgntn nextArw fr']")
            next_page.click()
            time.sleep(5)
            page_number += 1
            print(page_number)
        except:
            break


url = 'https://www.kohls.com/catalog/womens-clothing.jsp?CN=Gender:Womens+Department:Clothing&icid=wo-size-womens&kls_sbp=72721882801325196462708528859614168656'
browser.get(url)
browser.execute_script("window.scrollBy(0,250);")


scraped_data = ScrapePage(browser, url)
print(scraped_data)



None


In [36]:
print(item_list)


["Women's Gloria Vanderbilt Amanda Classic Jeans", '$48.00', "Women's Sonoma Goods For Life® Supersoft Midrise ...", '$12.60 - $24.99', "Women's Sonoma Goods For Life® Supersoft Stretch ...", '$12.60 - $24.99', "Women's Sonoma Goods For Life® Straight-Leg ...", '$16.20 - $24.99', "Women's Croft & Barrow® Effortless Stretch Pull-On ...", '$34.99', "Women's Lee® Wrinkle-Free Relaxed Fit Straight-Leg ...", '$44.00', "Women's Sonoma Goods For Life® Midrise Bootcut Jeans", '$24.99', 'Plus Size Gloria Vanderbilt Amanda Classic Jeans', '$52.00', "Women's Lee® Relaxed Fit Straight-Leg Jeans", '$44.00', "Women's Levi's® Classic Straight-Leg Jeans", '$48.65', 'Bali® Comfort Revolution® ComfortFlex Fit® Shaping ...', '$36.00', "Women's Tek Gear® Ultrasoft Fleece Sweatshirt", '$22.99', "Women's Sonoma Goods For Life® Midrise Leggings", '$11.99', "Women's Nine West Magic Waist Barely Bootcut Pants", '$38.99', "Women's Lee® Relaxed Fit Straight-Leg Twill Pants", '$44.00', "Women's Sonoma Goods For L

In [34]:
len(item_list)

96

In [67]:
print(clothing_lst)

["Women's Gloria Vanderbilt Amanda Classic Jeans", "Women's Sonoma Goods For Life® Supersoft Midrise ...", "Women's Sonoma Goods For Life® Supersoft Stretch ...", "Women's Sonoma Goods For Life® Straight-Leg ...", "Women's Croft & Barrow® Effortless Stretch Pull-On ...", "Women's Lee® Wrinkle-Free Relaxed Fit Straight-Leg ...", "Women's Sonoma Goods For Life® Midrise Bootcut Jeans", 'Plus Size Gloria Vanderbilt Amanda Classic Jeans', "Women's Lee® Relaxed Fit Straight-Leg Jeans", "Women's Levi's® Classic Straight-Leg Jeans", 'Bali® Comfort Revolution® ComfortFlex Fit® Shaping ...', "Women's Tek Gear® Ultrasoft Fleece Sweatshirt", "Women's Sonoma Goods For Life® Midrise Leggings", "Women's Nine West Magic Waist Barely Bootcut Pants", "Women's Lee® Relaxed Fit Straight-Leg Twill Pants", "Women's Sonoma Goods For Life® Curvy High-Waisted ...", "Women's Croft & Barrow® Classic Pull-On Jeans", "Women's Gloria Vanderbilt Amanda Pull-On Jeans", "Women's Lee® Instantly Slims High Waisted ...",

In [57]:
print(listed_price_lst)

['$48.00', '$12.60 - $24.99', '$12.60 - $24.99', '$16.20 - $24.99', '$34.99', '$44.00', '$24.99', '$52.00', '$44.00', '$48.65', '$36.00', '$22.99', '$11.99', '$38.99', '$44.00', '$24.99', '$27.99', '$54.00', '$44.00', '$34.99', '$48.65', '$39.99', '$48.00', '$21.60', '$37.50', '$11.99', '$29.99', '$6.50 - $11.99', '$10.80 - $12.99', '$29.25', '$22.00 - $33.00', '$36.00', '2 / $20.00', '$22.99', '$29.25', '$34.99', '$19.99', '$44.99', '$34.99', '$22.99', '$24.99', '$27.99', '$7.20 - $9.99', '$29.99', '$25.99', '$55.65', '$26.99', '$32.99']


In [60]:
Macys_series = pd.Series(listed_price_lst,index = clothing_lst)

In [61]:
Macys_series.to_csv('Macys_series.csv')

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

url = "https://www.macys.com/shop/womens-clothing/all-womens-clothing?id=188851&cm_sp=us_hdr-_-women-_-188851_all-women%\27s-clothing_COL1"

# initialize the web driver
driver = webdriver.Chrome()

# navigate to the website
driver.get(url)

# find all the products on the page
products = driver.find_elements(By.CLASS_NAME, "productDetail")


# iterate through the product links and extract relevant information
for product in products:
    product_name = products.find_element(By.CLASS_NAME, "productDescription").text
    product_price = products.find_element(By.CLASS_NAME, "priceInfo").text
    print(product_name, product_price)

#write the data to a csv file
with open("macys_data.csv", "w") as f:
    f.write("product_name,product_price")

    

    

# close the web driver
driver.quit()


AttributeError: 'list' object has no attribute 'find_element'

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.macys.com/shop/womens-clothing?id=21122"

# initialize the web driver
driver = webdriver.Chrome()

# navigate to the website
driver.get(url)

# wait for the product links to appear on the page
wait = WebDriverWait(driver, 10)
product_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".productDescLink")))

# iterate through the product links and extract relevant information
for link in product_links:
    product_name = link.find_element(By.CLASS_NAME, ("title")).text
    
    product_price = link.find_element(By.CSS_SELECTOR, ".productPrice").text
    print(product_name, product_price)

# close the web driver
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"title"}
  (Session info: chrome=109.0.5414.119)
Stacktrace:
0   chromedriver                        0x0000000103491fa8 chromedriver + 4886440
1   chromedriver                        0x000000010340f643 chromedriver + 4351555
2   chromedriver                        0x000000010305db27 chromedriver + 477991
3   chromedriver                        0x00000001030a276c chromedriver + 759660
4   chromedriver                        0x00000001030a29c1 chromedriver + 760257
5   chromedriver                        0x0000000103094506 chromedriver + 701702
6   chromedriver                        0x00000001030c85fd chromedriver + 914941
7   chromedriver                        0x00000001030943dd chromedriver + 701405
8   chromedriver                        0x00000001030c878e chromedriver + 915342
9   chromedriver                        0x00000001030e35fe chromedriver + 1025534
10  chromedriver  